In [4]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 kB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.6.0-py2.py3-none-any.whl size=351311 sha256=d87be21df61cff6dafa8e40dce1fea4a200d080bba0c7d5f7bd63b86a4f7d9d8
  Stored in directory: /root/.cache/pip/wheels/ea/0b/64/114bc939d0083621aa41521e21be246c888260b8aa21e6c1ad
Successfully built emoji


In [1]:
!pip install nltk

In [2]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.0 MB/s eta 0:00:00


In [16]:
import pandas as pd
import re
import emoji

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.util import mark_negation
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import contractions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
headers = ["id", "company", "sentiment", "tweet"]
df_train = pd.read_csv('/content/twitter_training.csv', names=headers)
df_val = pd.read_csv('/content/twitter_validation.csv', names=headers)

print(df_train.head())
print(df_val.head())

     id      company sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company   sentiment  \
0  3364   Facebook  Irrelevant   
1   352     Amazon     Neutral   
2  8312  Microsoft    Negative   
3  4371      CS-GO    Negative   
4  4433     Google     Neutral   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [10]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [11]:
# Step 1: Remove NaN values
def remove_nan_values(df, columns):
    for column in columns:
        df = df[df[column].notna()]  # Remove rows with NaN values
        df = df[df[column].astype(bool)]  # Remove rows with empty lists or lists with only white spaces
    return df

# Step 2: Assign tweet labels
def one_hot_encode_sentiments(sentiment):
    if sentiment == 'Positive':
        return 1
    elif sentiment == 'Negative':
        return -2
    elif sentiment == 'Neutral':
        return 0
    elif sentiment == 'Irrelevant':
        return -1
    else:
      return '?><?'

def expand_slang(text):
    tokens = word_tokenize(text)
    tokens = [abbreviations.get(word.lower(), word) for word in tokens]
    text = ' '.join(tokens)
    return text


# Step 3: Clean tweets
def clean_tweets(tweet):
    # Remove @mentions
    tweet = re.sub(r'@\w+', '', tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+', '', tweet)

    # Remove hashtags
    tweet = re.sub(r'#\w+', '', tweet)

    # Convert emojis to text
    tweet = emoji.demojize(tweet)

    # Expand contractions
    tweet = contractions.fix(tweet)
    ##############################3

    # Remove numbers
    tweet = re.sub(r'\d+', '', tweet)

    # Remove punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Remove excessive whitespace
    tweet = re.sub(r'\s+', ' ', tweet)

    # Standardize repeated characters
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)

    #Expand slang terms
    tweet = expand_slang(tweet)

    # Tokenize the tweet
    tokens = word_tokenize(tweet)

    # Lowercase and normalize
    tokens = [token.lower() for token in tokens]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Handle abbreviations and acronyms
    abbreviation_words = {"lol": "laughing out loud", "btw": "by the way"}  # Add more as needed
    tokens = [abbreviation_words.get(token.lower(), token) for token in tokens]

    # Handle negation using mark_negation
    #negation_tokens = mark_negation(tokens)
    #tweet = "I don't like this product. It is not good."
    #['I', "don't", 'like_NEG', 'this_NEG', 'product_NEG', '.', 'It_NEG', 'is_NEG', 'not_NEG', 'good_NEG', '.']

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Uncomment the following lines to include POS tags as comments
    #pos_tags = nltk.pos_tag(tokens)
    #tokens_with_pos = [f"{token}/{pos}" for token, pos in pos_tags]

    return tokens

In [12]:
# Step 1: Remove NaN values
columns_to_check = ["id", "company", "sentiment", "tweet"]
df_train = remove_nan_values(df_train, columns_to_check)
df_val = remove_nan_values(df_val, columns_to_check)

print(df_train.head())
print(df_val.head())

     id      company sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company   sentiment  \
0  3364   Facebook  Irrelevant   
1   352     Amazon     Neutral   
2  8312  Microsoft    Negative   
3  4371      CS-GO    Negative   
4  4433     Google     Neutral   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [13]:
# Step 2: Assign tweet labels
df_train['sentiment'] = df_train['sentiment'].apply(one_hot_encode_sentiments)
df_val['sentiment'] = df_val['sentiment'].apply(one_hot_encode_sentiments)

print(df_train.head())
print(df_val.head())

     id      company  sentiment  \
0  2401  Borderlands          1   
1  2401  Borderlands          1   
2  2401  Borderlands          1   
3  2401  Borderlands          1   
4  2401  Borderlands          1   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company  sentiment  \
0  3364   Facebook         -1   
1   352     Amazon          0   
2  8312  Microsoft         -2   
3  4371      CS-GO         -2   
4  4433     Google          0   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [17]:
# Step 3: Clean tweets
df_train['tweet'] = df_train['tweet'].apply(clean_tweets)
df_val['tweet'] = df_val['tweet'].apply(clean_tweets)

print(df_train.head())
print(df_val.head())

     id      company  sentiment                          tweet
0  2401  Borderlands          1  [getting, borderland, murder]
1  2401  Borderlands          1         [coming, border, kill]
2  2401  Borderlands          1    [getting, borderland, kill]
3  2401  Borderlands          1   [coming, borderland, murder]
4  2401  Borderlands          1  [getting, borderland, murder]
     id    company  sentiment  \
0  3364   Facebook         -1   
1   352     Amazon          0   
2  8312  Microsoft         -2   
3  4371      CS-GO         -2   
4  4433     Google          0   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
1  [british, broadcasting, corporation, news, ama...  
2  [pay, word, function, poorly, chromebook, face...  
3  [csgo, matchmaking, full, closet, hacking, tru...  
4  [president, slapping, american, face, really, ...  


In [ ]:
import pandas as pd
from collections import Counter

# Assuming you have a DataFrame named df_train with columns 'sentiment' and 'tweet'

# Step 1: Collect the tweets and sentiment encodings from the DataFrame
tweets = df_train['tweet'].tolist()
sentiments = df_train['sentiment'].tolist()

# Step 3: Combine the tweets with their corresponding sentiment encodings into a list of tuples
tweet_sentiment_pairs = list(zip(tweets, sentiments))

# Step 4: Count the frequency of each word for each sentiment
word_frequencies = {
    tuple([1, 0, 0, 0]): Counter(),
    tuple([0, 1, 0, 0]): Counter(),
    tuple([0, 0, 1, 0]): Counter(),
    tuple([0, 0, 0, 1]): Counter()
}

for tweet, sentiment in tweet_sentiment_pairs:
    word_frequencies[tuple(sentiment)].update(tweet)

# Step 5: Determine the most frequent words for each sentiment
most_frequent_words = {
    tuple([1, 0, 0, 0]): [],
    tuple([0, 1, 0, 0]): [],
    tuple([0, 0, 1, 0]): [],
    tuple([0, 0, 0, 1]): []
}
infrequent_words = set()

for sentiment, counter in word_frequencies.items():
    most_common_words = counter.most_common()
    most_frequent_words[sentiment] = most_common_words

    # Step 6: Handle infrequent words
    frequency_threshold = 5  # Adjust this threshold as needed
    infrequent_words.update(word for word, frequency in most_common_words if frequency <= frequency_threshold)

# Print the most frequent words for each sentiment
sentiment_mapping = {
    tuple([1, 0, 0, 0]): 'Positive',
    tuple([0, 1, 0, 0]): 'Negative',
    tuple([0, 0, 1, 0]): 'Neutral',
    tuple([0, 0, 0, 1]): 'Irrelevant'
}

for sentiment, words in most_frequent_words.items():
    print(f"Most frequent words for sentiment {sentiment_mapping[sentiment]}:")
    for word, frequency in words:
        print(f"{word}: {frequency}")
    print()

# Example of handling infrequent words
print("Infrequent words:")
print(infrequent_words)

Streaming output truncated to the last 5000 lines.
napc: 3
twitchtvmilzz_ttd: 3
fanfics: 3
tmblrcozaepwyrtevye: 3
newsyahoocomuspsfanslovi: 3
opendlabcomproject: 3
cakpepfefq: 3
joshimuz: 3
gtasan: 3
pictwittercomrelnugs: 3
optimised: 3
repetitive: 3
murdering: 3
realted: 3
twittercomdabi_hawkssta: 3
cultfan: 3
keeling: 3
twittercommylsbckstat: 3
mylsbck: 3
ponti: 3
motherboards: 3
stroke: 3
sikkim: 3
shri: 3
coverage: 3
guampdncomstorymoney: 3
pictwittercompsjmgzopb: 3
sympathy: 3
leveraging: 3
pictwittercomswvnohfv: 3
glancing: 3
herbsthe: 3
youtubedekfjls_e: 3
homicidecanadacomtorontoh: 3
homespublishin: 3
canadiangta: 3
passemuraillecaseasons: 3
eine: 3
villa: 3
twitchtvpwerflyy: 3
pictwittercomqanftkqsog: 3
whine: 3
disclose: 3
tonesfor: 3
bullshitkill: 3
pictwittercomkevzr: 3
twittercomapollontweets: 3
bitlyfdkk: 3
hellcomputerscom: 3
twittercommaxshotostatu: 3
pictwittercomxyzaggqh: 3
wdlegion: 3
timed: 3
legion: 3
pictwittercomeqpzkyt: 3
fangamer: 3
rushed: 3
squareenixgamescom

In [18]:
# Step 1: Remove NaN values
columns_to_check = ["id", "company", "sentiment", "tweet"]
df_train = remove_nan_values(df_train, columns_to_check)
df_val = remove_nan_values(df_val, columns_to_check)

print(df_train.head())
print(df_val.head())

     id      company  sentiment                          tweet
0  2401  Borderlands          1  [getting, borderland, murder]
1  2401  Borderlands          1         [coming, border, kill]
2  2401  Borderlands          1    [getting, borderland, kill]
3  2401  Borderlands          1   [coming, borderland, murder]
4  2401  Borderlands          1  [getting, borderland, murder]
     id    company  sentiment  \
0  3364   Facebook         -1   
2  8312  Microsoft         -2   
3  4371      CS-GO         -2   
5  6273       FIFA         -2   
6  7925  MaddenNFL          1   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
2  [pay, word, function, poorly, chromebook, face...  
3  [csgo, matchmaking, full, closet, hacking, tru...  
5  [hi, madeleine, mccann, cellar, past, year, li...  
6  [thank, new, te, austin, hooper, orange, brown...  


In [19]:
# Print the resulting DataFrame
print(df_train.head())
print(df_val.head())

     id      company  sentiment                          tweet
0  2401  Borderlands          1  [getting, borderland, murder]
1  2401  Borderlands          1         [coming, border, kill]
2  2401  Borderlands          1    [getting, borderland, kill]
3  2401  Borderlands          1   [coming, borderland, murder]
4  2401  Borderlands          1  [getting, borderland, murder]
     id    company  sentiment  \
0  3364   Facebook         -1   
2  8312  Microsoft         -2   
3  4371      CS-GO         -2   
5  6273       FIFA         -2   
6  7925  MaddenNFL          1   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
2  [pay, word, function, poorly, chromebook, face...  
3  [csgo, matchmaking, full, closet, hacking, tru...  
5  [hi, madeleine, mccann, cellar, past, year, li...  
6  [thank, new, te, austin, hooper, orange, brown...  


In [21]:
# Assuming your DataFrame is named df

# Drop the 'id' column
df_train_new = df_train.drop('id', axis=1)
df_val_new = df_val.drop('id', axis=1)

# Export the DataFrame to a CSV file
df_train_new.to_csv('df_train_new_int.csv', index=False)
df_val_new.to_csv('df_val_new_int.csv', index=False)
